## Introduction
: Lag features require that the lagged target value is known at the time being forecast. A lag 1 feature shifts the time series forward 1 step, which means you could forecast 1 step into the future but not 2 steps.

So in this lesson, we'll learn how to make forecasts for a variety of situations.

## Defining the Forecasting Task<br>

Before designing a forecasting model, check
* what information is available at the time a forecast is made (features), and,
* the time period during which you require forecasted values (target).<br>
        * **Forecast origin** : The time at which you are making a forecast. Usually, it's the last time for which you have training data for the time being predicted. Everything up to he origin can be used to create features.<br>
        * **Forecast horizon** : The time for which you are making a forecast. Usually described by the number of time steps. The forecast horizon describes the target.

<img src="06_defining a forecast.png" alt="title" width="500"/>

## Preparing Data for Forecasting

* Transform the series into a dataframe if needed
* Created a feature set out of lags. 
* Preparing the target. (How we do this depends on the forecasting task.)

## Multistep Forecasting Strategies¶

* **Multioutput model**<br>
: Use a model that produces multiple outputs naturally. (e.g. Linear regression, neural networks) Simple and efficient, but not possible for every algorithm you might want to use. (e.g. XGBoost can't do this)<br>
* **Direct strategy**<br>
: Train a separate model for each step in the horizon: one model forecasts 1-step ahead, another 2-steps ahead, and so on. It can help to have a different model make forecasts for each step, but computationally expensive.<br>
* **Recursive strategy**<br>
: Train a single one-step model and use its forecasts to update the lag features for the next step. We only need to train one model, but since errors will propagate from step to step, forecasts can be inaccurate for long horizons.<br>
* **DirRec strategy**<br>
: A combination of the direct and recursive strategies: train a model for each step and use forecasts from previous steps as new lag features. Step by step, each model gets an additional lag input. Since each model always has an up-to-date set of lag features, the DirRec strategy can capture serial dependence better than Direct, but it can also suffer from error propagation like Recursive.

## Excerise Notes

* Examples
1) 1-step forecast using 3 lag features with a 1-step lead time<br>
<img src="06_task1.png" alt="title" width="220"/><br>

2) 3-step forecast using 4 lag features with a 2-step lead time<br>
<img src="06_task2.png" alt="title" width="380"/><br>

3) 3-step forecast using 4 lag features with a 1-step lead time<br>
<img src="06_task3.png" alt="title" width="380"/>

* Creat targets (4 days of lag features. Drop any missing values from both targets and features.)<br>
    ```python
    y = family_sales.loc[:, 'sales']

    # Make 4 lag features and drop missing values
    X = make_lags(y, lags=4).dropna

    # Make multistep target and drop missing values
    y = make_multistep_target(y, steps=16).dropna()

    y, X = y.align(X, join='inner', axis=0)
    ```

* Forecast with the DirRec strategy¶
    ```python
    from sklearn.multioutput import RegressorChain
    # Instatiate a model that applies the DirRec strategy to XGBoost.
    model = RegressorChain(base_estimator=XGBRegressor())
    ```